In [1]:
# Импортирование необходимых библиотек, по ходу будут добавлятся другие
import pandas as pd
import numpy as np

In [2]:
# Импорт данных
df=pd.read_csv("data/dataset_clusterd.csv", sep="\t", encoding="utf-16")

In [3]:
df['значимость_раздела_wsss'] = pd.to_numeric(df['значимость_раздела_wsss'],errors='coerce')
df["значимость_раздела_wsss"]=df["значимость_раздела_wsss"].fillna(float(int(df["значимость_раздела_wsss"].mean())))
df.drop(df.iloc[:,8:],inplace=True,axis=1)
df.head()

,название_компетенции_wsss,описание_компетенции_wsss,название_раздела_wsss,значимость_раздела_wsss,признаки_раздела_wsss,cluster,nameed_clusters
0,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,обработка поверхности,15.0,специалист должен знать понимать специализиров...,0,Информационные и коммуникационные технологии
1,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,изготовление сложных элементов ювелирных издел...,20.0,специалист должен знать понимать способы испол...,0,Информационные и коммуникационные технологии
2,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,изготовление простых элементов ювелирных изделий,20.0,специалист должен знать понимать способы испол...,0,Информационные и коммуникационные технологии
3,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,подготовка драгоценных сплавов изготовления эл...,10.0,специалист должен знать понимать свойства спос...,0,Информационные и коммуникационные технологии
4,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,изготовление сплавов драгоценных металлов,5.0,специалист должен знать понимать состав сплаво...,0,Информационные и коммуникационные технологии


In [4]:
# Рандомизация данных
df=df.sample(n=len(df))

In [5]:
# Трансформаторы для получения данных
from sklearn.base import BaseEstimator, TransformerMixin
# Конвеер для объединения шагов выполнения классификации
from sklearn.pipeline import Pipeline
# Функция стандартизации числовых значений
from sklearn.preprocessing import StandardScaler
# Функция преобразования текстовых данных под модель
from sklearn.feature_extraction.text import TfidfVectorizer

# Класс необходимый для получения и работы с текстовыми признаками
class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.key]
    
# Класс необходимый для получения работы с числовыми признаками
class NumericalSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[[self.key]]

In [6]:
# Стеммер, работающий с русским языком
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian") 

# Функция стеммизации данных для образования однокоренных слов
def stemming_tokenizer(text):
    text = re.split('\W+', text)
    text = [stemmer.stem(word) for word in text]
    return text

In [7]:
# Получение стоп-слов из библиотеки nltk(и, за, а, на, как и.т.) 
from nltk.corpus import stopwords
import re
stop_words = stopwords.words("russian")

In [8]:
df.head()

,название_компетенции_wsss,описание_компетенции_wsss,название_раздела_wsss,значимость_раздела_wsss,признаки_раздела_wsss,cluster,nameed_clusters
482,Промышленная автоматика,промышленная автоматика включает элементы элек...,поиск неисправностей,10.0,специалист должен знать понимать требования бе...,0,Информационные и коммуникационные технологии
1193,Графический дизайн,графическим дизайном понимается множество комп...,решение проблем,12.0,специалист должен знать понимать общие проблем...,0,Информационные и коммуникационные технологии
35,Управление фронтальным погрузчиком,фронтальный погрузчик назвать универсальной ма...,ремонт агрегатов систем фронтального погрузчика,28.0,участник должен знать понимать основные сведен...,0,Информационные и коммуникационные технологии
557,Электроника,индустрия электроники является разнообразной о...,сборка,20.0,специалист должен знать понимать соответствующ...,0,Информационные и коммуникационные технологии
322,Кузовной ремонт,мастер кузовного ремонта ремонтирует структурн...,замена деталей панелей требующих сварку,34.0,специалист должен знать понимать важность след...,0,Информационные и коммуникационные технологии


In [10]:
# Подгонка признаков под конвейер для корректной работы с моделью
dictionary = Pipeline([
    ('selector', TextSelector(key="описание_компетенции_wsss")),
    ('tfidf', TfidfVectorizer(stop_words = stop_words,tokenizer=stemming_tokenizer))
    ])

dictionary.fit_transform(df).todense()

E:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['бол', 'больш', 'будт', 'быт', 'вед', 'впроч', 'всег', 'всегд', 'даж', 'друг', 'е', 'ег', 'ем', 'есл', 'ест', 'ещ', 'зач', 'зде', 'ил', 'иногд', 'когд', 'конечн', 'куд', 'лучш', 'межд', 'мен', 'мног', 'мо', 'можн', 'нег', 'нельз', 'нибуд', 'никогд', 'нич', 'опя', 'посл', 'пот', 'почт', 'разв', 'сво', 'себ', 'совс', 'теб', 'тепер', 'тог', 'тогд', 'тож', 'тольк', 'хорош', 'хот', 'чег', 'чут', 'эт'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.09157595, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [11]:
# Подгонка признаков под конвейер для корректной работы с моделью
title = Pipeline([
    ('selector', TextSelector(key="название_раздела_wsss")),
    ('tfidf', TfidfVectorizer(stop_words = stop_words,tokenizer=stemming_tokenizer))
    ])

title.fit_transform(df).todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
# Подгонка признаков под конвейер для корректной работы с моделью
ves_title = Pipeline([
    ('selector', NumericalSelector(key="значимость_раздела_wsss")),
    ('tfidf', StandardScaler())
    ])

ves_title.fit_transform(df)

array([[-0.5112745 ],
       [-0.32734803],
       [ 1.1440637 ],
       ...,
       [ 0.89576297],
       [ 0.868174  ],
       [ 0.40835783]])

In [13]:
# Подгонка признаков под конвеер для корректной работы с моделью
title_feat = Pipeline([
    ('selector', TextSelector(key="признаки_раздела_wsss")),
    ('tfidf', TfidfVectorizer(stop_words = stop_words,tokenizer=stemming_tokenizer))
    ])

title_feat.fit_transform(df).todense()

E:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['бол', 'больш', 'будт', 'быт', 'вед', 'впроч', 'всег', 'всегд', 'даж', 'друг', 'е', 'ег', 'ем', 'есл', 'ест', 'ещ', 'зач', 'зде', 'ил', 'иногд', 'когд', 'конечн', 'куд', 'лучш', 'межд', 'мен', 'мног', 'мо', 'можн', 'нег', 'нельз', 'нибуд', 'никогд', 'нич', 'опя', 'посл', 'пот', 'почт', 'разв', 'сво', 'себ', 'совс', 'теб', 'тепер', 'тог', 'тогд', 'тож', 'тольк', 'хорош', 'хот', 'чег', 'чут', 'эт'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [14]:
df.head()

,название_компетенции_wsss,описание_компетенции_wsss,название_раздела_wsss,значимость_раздела_wsss,признаки_раздела_wsss,cluster,nameed_clusters
482,Промышленная автоматика,промышленная автоматика включает элементы элек...,поиск неисправностей,10.0,специалист должен знать понимать требования бе...,0,Информационные и коммуникационные технологии
1193,Графический дизайн,графическим дизайном понимается множество комп...,решение проблем,12.0,специалист должен знать понимать общие проблем...,0,Информационные и коммуникационные технологии
35,Управление фронтальным погрузчиком,фронтальный погрузчик назвать универсальной ма...,ремонт агрегатов систем фронтального погрузчика,28.0,участник должен знать понимать основные сведен...,0,Информационные и коммуникационные технологии
557,Электроника,индустрия электроники является разнообразной о...,сборка,20.0,специалист должен знать понимать соответствующ...,0,Информационные и коммуникационные технологии
322,Кузовной ремонт,мастер кузовного ремонта ремонтирует структурн...,замена деталей панелей требующих сварку,34.0,специалист должен знать понимать важность след...,0,Информационные и коммуникационные технологии


In [15]:
from sklearn.pipeline import FeatureUnion
# Формирование конвейера 
feat = FeatureUnion([("title", title),
                    ("ves_title", ves_title),
                    ("title_feat", title_feat)])

# Переменная, которая пойдёт на разделение набора и последующее обучкение и тестирование
X=feat.fit_transform(df).todense()

In [16]:
from sklearn.model_selection import train_test_split #Формирование конвейера
# Целевая переменная, которая содержит названия кластеров
y=df["nameed_clusters"]
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.1)

In [17]:
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

In [18]:
from sklearn.metrics import classification_report

In [20]:
# Метод SVC(Метод опорных векторов)   #Обучение моделей и их сравнение
pipeline_svc=SVC()

# Обучение классификатора на обучающих данных
pipeline_svc.fit(X_train,y_train)
# Предсказание целевой переменной на тестирующих данных
preds_scv=pipeline_svc.predict(X_test)
# Вывод со всеми описанными метриками ранее
print(classification_report(y_test, preds_scv))

                                              precision    recall  f1-score   support

Информационные и коммуникационные технологии       0.96      1.00      0.98       129
        Производство и инженерные технологии       1.00      1.00      1.00         2
     Строительство и строительные технологии       1.00      0.81      0.90        16
                                 Сфера услуг       1.00      0.87      0.93        15

                                    accuracy                           0.97       162
                                   macro avg       0.99      0.92      0.95       162
                                weighted avg       0.97      0.97      0.97       162



In [21]:
# Метод GaussianNB
pipeline_gnb=GaussianNB()    #Метод GaussianNB показал точность: 0.79

# Обучение классификатора на обучающих данных
pipeline_gnb.fit(X_train,y_train)
# Предсказание целевой переменной на тестирующих данных
preds_gnb=pipeline_gnb.predict(X_test)
# Вывод со всеми описанными метриками ранее
print(classification_report(y_test, preds_gnb))

                                              precision    recall  f1-score   support

Информационные и коммуникационные технологии       1.00      0.96      0.98       129
        Производство и инженерные технологии       1.00      1.00      1.00         2
     Строительство и строительные технологии       0.80      1.00      0.89        16
                                 Сфера услуг       0.94      1.00      0.97        15

                                    accuracy                           0.97       162
                                   macro avg       0.93      0.99      0.96       162
                                weighted avg       0.97      0.97      0.97       162



In [23]:
# Метод LogisticRegression
pipeline_lr=LogisticRegression() #Метод LogisticRegression показал точность: 0.83

# Обучение классификатора на обучающих данных
pipeline_lr.fit(X_train,y_train)
# Предсказание целевой переменной на тестирующих данных
preds_lr=pipeline_lr.predict(X_test)
# Вывод со всеми описанными метриками ранее
print(classification_report(y_test, preds_lr))

                                              precision    recall  f1-score   support

Информационные и коммуникационные технологии       0.96      1.00      0.98       129
        Производство и инженерные технологии       1.00      1.00      1.00         2
     Строительство и строительные технологии       0.92      0.75      0.83        16
                                 Сфера услуг       1.00      0.80      0.89        15

                                    accuracy                           0.96       162
                                   macro avg       0.97      0.89      0.92       162
                                weighted avg       0.96      0.96      0.95       162

